## p53 network - Metropolis Hastings

In [ ]:
from IPython.display import display
import matplotlib.pyplot as plt

from reggae.data_loaders import load_barenco_puma, DataHolder, scaled_barenco_data
from reggae.mcmc import create_chains, MetropolisHastings
from reggae.models import transcription_mh
from reggae.utilities import discretise, inverse_positivity
from reggae.plot import plotters
from reggae.models import Options
from reggae.models.results import GenericResults, SampleResultsMH

import numpy as np
import pandas as pd
import tensorflow as tf
import arviz
from multiprocessing import Pool

np.set_printoptions(formatter={'float': lambda x: "{0:0.5f}".format(x)})
plt.style.use('ggplot')
%matplotlib inline

In [ ]:
#df, genes, genes_se, m_observed, f_observed, σ2_m_pre, σ2_f_pre, t = load_barenco_puma()
m_observed, f_observed, σ2_m_pre, σ2_f_pre, t = load_barenco_puma()

# m_observed, f_observed, t = load_3day_dros()

replicate = 0

m_df, m_observed = m_observed 
f_df, f_observed = f_observed

data_full = (m_observed, f_observed)

# Shape of m_observed = (replicates, genes, times)
m_observed = m_observed[replicate]
f_observed = np.atleast_2d(f_observed[replicate])
σ2_m_pre = σ2_m_pre[0]
σ2_f_pre = σ2_f_pre[0]

num_genes = m_observed.shape[0]
τ, common_indices = discretise(t)
N_p = τ.shape[0]
N_m = m_observed.shape[1]

data = (m_observed, f_observed)
noise_data = (σ2_m_pre, σ2_f_pre)
time = (t, τ, tf.constant(common_indices))

data = DataHolder(data, noise_data, time)
opt = Options(preprocessing_variance=True, tf_mrna_present=True)

data_full = DataHolder(data_full, noise_data, time)


In [ ]:
T = 100
store_every = 1
burn_in = 0
report_every = 20
num_chains = 4
tune_every = 50

job = create_chains(
    transcription_mh.TranscriptionMCMC, 
    [data, opt], 
    {
        'T': T, 
        'store_every': store_every, 
        'burn_in': burn_in,
        'report_every': report_every,
        'tune_every':tune_every
    }, 
    num_chains=num_chains)

    
print('Done')

## Convergence Plots

In [ ]:
keys = job[0].acceptance_rates.keys()

variables = {key : np.empty((0, T, *job[0].samples[key].get().shape[1:])) for key in keys}

for res in job:
    for key in keys:
        variables[key] = np.append(variables[key], np.expand_dims(res.samples[key].get(), 0), axis=0)

plt.plot(variables['L'][:,-100:].T)

mixes = {key: arviz.convert_to_inference_data(variables[key]) for key in keys}

#### Rhat
Rhat is the ratio of posterior variance and within-chain variance. If the ratio exceeds 1.1 then we consider the chains have not mixed well. As the between-chain variance tends to the within-chain then R tends to 1.

In [ ]:
Rhat = arviz.rhat(mixes['fbar'])

Rhats = np.array([np.mean(arviz.rhat(mixes[key]).x.values) for key in keys])

rhat_df = pd.DataFrame([[*Rhats], [*(Rhats < 1.1)]], columns=keys)

display(rhat_df)

#### Rank plots

Rank plots are histograms of the ranked posterior draws (ranked over all
    chains) plotted separately for each chain.
    If all of the chains are targeting the same posterior, we expect the ranks in each chain to be
    uniform, whereas if one chain has a different location or scale parameter, this will be
    reflected in the deviation from uniformity. If rank plots of all chains look similar, this
    indicates good mixing of the chains.

Rank-normalization, folding, and localization: An improved R-hat
    for assessing convergence of MCMC. arXiv preprint https://arxiv.org/abs/1903.08008

In [ ]:
arviz.plot_rank(mixes['L'])

#### Effective sample sizes

Plot quantile, local or evolution of effective sample sizes (ESS).

In [ ]:
arviz.plot_ess(mixes['L'])

#### Monte-Carlo Standard Error

In [ ]:
arviz.plot_mcse(mixes['L'])


#### Parallel Plot
Plot parallel coordinates plot showing posterior points with and without divergences.

Described by https://arxiv.org/abs/1709.01449, suggested by Ari Hartikainen


In [ ]:
arviz.plot_parallel(azl)


Step size is standard dev, too small means it takes long time to reach high density areas. too long means we reject many of samples

## Plots

In [ ]:
samples = job[0].samples
acceptance_rates = job[0].acceptance_rates

model = transcription_mh.TranscriptionMCMC.initialise_from_state([data, opt], job[0])

In [ ]:
# Begin MCMC
model = transcription_mh.TranscriptionMCMC(data, opt)
T = 200
store_every = 1
burn_in = 0
report_every = 1

model.sample(T, store_every, burn_in, report_every)

samples = model.samples
parameter_names = model.acceptance_rates.keys()

display(pd.DataFrame([[f'{100*samples["acc_rates"][name].get()[-1][0]:.02f}%' for name in parameter_names]], columns=parameter_names))


In [ ]:
## samples = transcription_model.samples
plt.figure(figsize=(10,14))
parameter_names = model.acceptance_rates.keys()
acc_rates = samples['acc_rates']

for i, name in enumerate(parameter_names):
    plt.subplot(len(parameter_names), 3, i+1)
    deltas = acc_rates[name].get()
    plt.plot(deltas)
    plt.title(name)
plt.tight_layout()

### Plot transcription ODE kinetic params


In [ ]:
σ2_f = None
σ2_m = samples['σ2_m'].get()
kbar =   samples['kbar'].get()
k_fbar = np.expand_dims(samples['δbar'].get(), [1, 2])
fbar =   samples['fbar'].get()
kernel_params = [samples[param].get() for param in ['L', 'V']]
w =      samples['w'].get()
w_0 =    samples['w_0'].get()

δbar = samples['δbar'].get()
m_preds = list()
for i in range(1, 20):
    m_preds.append(model.likelihood.predict_m(kbar[-i], δbar[-i], w[-1], fbar[-i], w_0[-1])) #todo w[-1]
m_preds = np.expand_dims(np.array(m_preds), 1)

fbar = np.expand_dims(fbar, [1, 2])
results = SampleResultsMH(fbar, kbar, k_fbar, None, kernel_params, w, w_0, σ2_m, σ2_f)

plotters.generate_report(data_full, results, m_preds, plot_barenco=True)